# Creating Environment

In [ ]:
import numpy as np
import numpy.random as rand

In [ ]:
class Cell():
    target = 0
    cell_type = ""
    prob_not_found = 0
    def __init__(self, cell_type, prob_not_found):
        self.cell_type = cell_type
        self.prob_not_found = prob_not_found
        

In [ ]:
#create environment
#take input from user
while True:
  try:
    dimension = int(input("Enter dimension of board: "))
    break
  except ValueError:
    print("Must be an integer")
    
mat = [[] for i in range(dimension)] # create a list of empty lists
cell_type = ""
prob_not_found = 0
for i in range(dimension):
    for j in range(dimension):
        p = rand.random()
        if p < 0.25:
            cell_type = "flat"
            prob_not_found = 0.1
        elif p <0.50:
            cell_type = "hilly"
            prob_not_found = 0.3
        elif p <0.75:
            cell_type = "forested"
            prob_not_found = 0.7
        else:
            cell_type = "maze of caves"
            prob_not_found = 0.9
        mat[i].append(Cell(cell_type, prob_not_found))
target = np.random.randint(dimension, size = 2)

Enter dimension of board: 2


In [ ]:
print(mat[0][0].cell_type)
print(mat[0][1].cell_type)
print(mat[1][0].cell_type)
print(mat[1][1].cell_type)
print(mat[1][0].prob_not_found)
print(target)

flat
forested
maze of caves
maze of caves
0.9
[0 1]


# Basic Agent 1

In [ ]:
def get_valid_neighbors(current_node):
  get_neighbors = [] #list of possible neighbors
  for i in range(-1,2):
    for j in range(-1,2):
      if i==0 and j==0:
        continue
      get_neighbors.append((i,j))
  x, y = current_node
  neighbors = set([])
  for i in get_neighbors:
    neighbor_x = x + i[0]
    neighbor_y = y + i[1]

    if neighbor_x >= dimension or neighbor_x < 0:
      continue
    elif neighbor_y >= dimension or neighbor_y < 0:
      continue
    neighbors.add((neighbor_x, neighbor_y))
  return neighbors

In [ ]:
def belief_state_initialize(dimension):
  belief_state = np.full((dimension,dimension),1/(dimension*dimension))
  return belief_state

In [ ]:
def search_current(current_cell, matrix, target):
  found_target = False
  if current_cell is target:
    if random.randrange(0, 1) > matrix[current_cell[0]][current_cell[1]].prob_not_found:
      found_target = True
  return found_target

In [ ]:
def Bayes_fn(current, belief_state, map):
    belief_in = belief_state[current[0]][current[1]]
    denom = belief_in * map[current[0]][current[1]].prob_not_found + (1 - belief_in) *1
    prob = belief_in * map[current[0]][current[1]].prob_not_found / denom
    return prob

In [ ]:
def Bayes_other(neighbor, current, belief_state, map):
    belief_in = belief_state[neighbor[0]][neighbor[1]]
    belief_curr = belief_state[current[0]][current[1]]
    marginalization = belief_curr * map[current[0]][current[1]].prob_not_found + (1 - belief_curr) *1
    prob = belief_in / marginalization
    return prob

In [ ]:
def basic_agent_1(map, dimension, target):
  #creating a search space map
  unsearched_spaces = set()
  for i in range(dimension):
    for j in range(dimension):
      unsearched_spaces.add((i,j))
  #starts at random point
  belief_state = belief_state_initialize(dimension)
  found_target = False
  current = np.random.randint(dimension, size = 2)
  print(current)
  path = set()
  path.add((current[0],current[1]))

  #loop when target isn't found

  #while found_target is False:
  for i in range(4):
    
    found_target = search_current(current, map, target)
    if found_target:
      break
    neighbors = get_valid_neighbors(current)
    max_belief = Bayes_fn(current, belief_state, map)
    belief_state[current[0]][current[1]] = max_belief
    next_node = np.random.randint(dimension, size = 2)
    #neighbors_probabilities = probability_neighbor_nodes(current, belief_state, map, neighbors)
    for i in neighbors:
      prob_other = Bayes_other(i, current, belief_state, map)
      belief_state[i[0]][i[1]] = prob_other
      if max_belief < prob_other:
        max_belief = prob_other
        next_node = (i[0],i[1])
    current = next_node
    #print(current)
    path.add((current[0],current[1]))
    print(current)
    print(belief_state)
    #print(path)
  return found_target

In [ ]:
print(mat[0][0].cell_type)
print(mat[1][0].cell_type)
print(mat[0][1].cell_type)
print(mat[1][1].cell_type)
print(target)

flat
maze of caves
forested
maze of caves
[0 1]


In [ ]:
path = basic_agent_1(mat, dimension, target)

[1 0]
(0, 1)
[[0.25590551 0.25590551]
 [0.23076923 0.25590551]]
(0, 0)
[[0.27172218 0.19402985]
 [0.24503231 0.27172218]]
(1, 1)
[[0.03596825 0.200521  ]
 [0.25322971 0.28081248]]
[0 1]
[[0.03692852 0.20587442]
 [0.25999033 0.26003329]]
